# How did the spreading of COVID-19 and the regulations in association with it influence the game market, in particular the game League of Legends, in terms of revenue, monthly player numbers, and popularity in Europe ?

As one knows, the spreading of the Coronavirus has lead most of the world into strong quarantine regulations. These regulations made everyone stay at home. Being made to stay at home, people started to take a bigger interest in finding new hobbies, experimenting new fields of interests and deepen there current hobbies.

Since leaving your house was an issue, all of these hobbies had to be indoor, therefore one of the booming industries next to the at-home-sports industry was the gaming market. People started to invest more into video games and sharpen their skills.
But how much was the gaming industry really impacted by these changes and during which time-period was the boom the strongest? Is it really true that thanks to COVID-19 the industry started rapidly growing again?

To find an answer to these questions we first had to collect COVID-19 data.
The Data we decided to collect was the following:
- Data of each month during the first COVID-19 caused quarantine
- Number of infections per Month in correlation of population in each European country

Therefore we had to find a Data Source , which is able to deliver that data to us. To do so we have researched several APIs, which can be of use. One of these APIs is Coronavirus COVID19 API (https://rapidapi.com/api-sports/api/covid-193).

This API provides up-to-date data about Coronavirus outbreak. Including numbers about confirmed cases, deaths and recovered.

Of course since COVID is a really recent topics there were other APIs to choose from as well, the problem with these were limited rate limits, country limitations (e.g.: only the US),

## COVID-19 Data

Before actually analyzing, visualizing, describing and working with the data we have to collect all the needed data.
As previously mentioned we want the data of the first every quarantine, for which the main timespan went from March 2020 til Mai 2020.

To get this Data we have to request it from our Coronavirus Tracker API. The results of these requests are going to be in json format, so we have to work with the json library as well.

In [21]:
import pandas as pd
import requests
import json
import datetime
import time

Since we want each country we are setting up a List, which contains all countries of europe.

In [27]:
# we need the country list
# country list is parted in three seperate lists to keep the rate limit low
europe_country_list = ["Albania", "Andorra", "Armenia", "Austria", "Azerbaijan", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Georgia", "Germany", "Greece", "Hungary", "Iceland", "Ireland", "Italy", "Kazakhstan", "Latvia", "Liechtenstein", "Lithuania", "Luxembourg", "Macedonia", "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", "Norway", "Poland","Portugal","Romania", "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "Turkey", "Ukraine","United Kingdom", "Vatican City"]

print(f"Number of Countries: {len(europe_country_list)}" )

Number of Countries: 50


In the next step we use this list ot gather the information per month and for each country.

In [23]:
url = "https://covid-193.p.rapidapi.com/history"

headers = {
	"X-RapidAPI-Key": "ba735a5542msh0672a6a248c9225p1ba64djsnfd32c8982360",
	"X-RapidAPI-Host": "covid-193.p.rapidapi.com"
}


### March Data

In [24]:
# collect the results for each country in that specific timeframe
# march
start_date = datetime.date(2020, 3, 1)
end_date = datetime.date(2020, 4, 1)
delta = datetime.timedelta(days=1)

for country in europe_country_list:
    j = []
    while start_date <= end_date:
        querystring = {"country":f"{country}","day":f"{start_date}"}
        result = requests.request("GET", url, headers=headers, params=querystring)
        country_json = result.json()
        # put it into the countries json
        j.append(country_json)
        start_date += delta
        # ensure that only 60 requests occur every minute (for api rate limits)
        time.sleep(1)
    # save the json
    with open(f"covid_data/api_data/march/{country}_march.json", "w") as jsonfile:
        json.dump(j, jsonfile)
    # set start date again
    start_date = datetime.date(2020, 3, 1)

KeyboardInterrupt: 

### April Data

In [ ]:
# april
start_date = datetime.date(2020, 4, 1)
end_date = datetime.date(2020, 5, 1)
delta = datetime.timedelta(days=1)

for country in europe_country_list:
    j = []
    while start_date <= end_date:
        querystring = {"country":f"{country}","day":f"{start_date}"}
        result = requests.request("GET", url, headers=headers, params=querystring)
        country_json = result.json()
        # put it into the countries json
        j.append(country_json)
        start_date += delta
        # ensure that only 60 requests occur every minute (for api rate limits)
        time.sleep(1)
    # save the json
    with open(f"covid_data/api_data/april/{country}_april.json", "w") as jsonfile:
        json.dump(j, jsonfile)
    start_date = datetime.date(2020, 4, 1)

### May Data

In [28]:
# may
start_date = datetime.date(2020, 5, 1)
end_date = datetime.date(2020, 6, 1)
delta = datetime.timedelta(days=1)

changed_list = europe_country_list[6:]

for country in changed_list:
    j = []
    while start_date <= end_date:
        querystring = {"country":f"{country}","day":f"{start_date}"}
        result = requests.request("GET", url, headers=headers, params=querystring)
        country_json = result.json()
        # put it into the countries json
        j.append(country_json)
        start_date += delta
        # ensure that only 60 requests occur every minute (for api rate limits)
        time.sleep(1)

    # save the json
    with open(f"covid_data/api_data/may/{country}_may.json", "w") as jsonfile:
        json.dump(j, jsonfile)
    start_date = datetime.date(2020, 5, 1)

## Saving the Data

## Visualizing Data

## Enrichment - Regulations
In Big Data it is important that the Data is not only collected, but that is also has some Value. To make Data valuable we have to enrich the data, meaning we bring some context to it. This can be achieved by forming a context as said. In our case this is going to mean that just the total number of cases is not enough. Next to the cases we need the regulations the countries have set during those specific months - Were bars closed? Were the clubs closed? Were people able to meet outside ? and so on.

Since these factors are also of great importance.

To do so we have found a .csv file which provides us with all the information concerning the countries regulations and how long these regulations were up and therefore had to be followed.
This data was found on the following website: https://www.ecdc.europa.eu/en/publications-data/download-data-response-measures-covid-19

In [ ]:
regulations_df = pd.read_csv("covid_data/country_regulations.csv")
regulations_df.head()